Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[ 0.3104, -0.3000,  0.0193, -0.0386,  0.2751,  0.0788],
        [ 0.0061, -0.0697,  0.2969, -0.2600,  0.0725,  0.3010],
        [ 0.2675, -0.1119, -0.3141, -0.1502,  0.0923, -0.2055],
        [ 0.1862, -0.2194,  0.2621, -0.2834, -0.2994, -0.3890],
        [-0.3968, -0.0334,  0.2835, -0.2249,  0.2566,  0.0784],
        [-0.3573,  0.2156,  0.0749,  0.3010, -0.1423, -0.2872],
        [ 0.0229,  0.1039,  0.2586,  0.1626,  0.0660, -0.1498],
        [-0.3115, -0.0155,  0.4040, -0.1845, -0.1167,  0.3602],
        [ 0.3515,  0.2266, -0.3906, -0.2857,  0.1716, -0.2755],
        [-0.0893, -0.1174,  0.3510,  0.1039, -0.2597, -0.0210],
        [-0.2963,  0.0514, -0.2917, -0.1297, -0.2724,  0.0975],
        [ 0.1478, -0.2056, -0.1820, -0.0112, -0.0986, -0.0405],
        [-0.1874, -0.0366, -0.1008, -0.3625, -0.3058, -0.1198],
        [-0.0580, -0.3050,  0.2990, -0.1933,  0.3246, -0.0087],
        [ 0.0723,  0.3429,  0.0716,  0.3635, -0.1066, -0.2327],
        [-0.0478, 

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[0.0618],
        [0.0615],
        [0.0646],
        ...,
        [0.0589],
        [0.0687],
        [0.0652]], grad_fn=<AddmmBackward0>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.0668)
1 tensor(0.0647)
2 tensor(0.0628)
3 tensor(0.0610)
4 tensor(0.0593)
5 tensor(0.0577)
6 tensor(0.0563)
7 tensor(0.0550)
8 tensor(0.0538)
9 tensor(0.0526)
10 tensor(0.0516)
11 tensor(0.0506)
12 tensor(0.0498)
13 tensor(0.0490)
14 tensor(0.0484)
15 tensor(0.0478)
16 tensor(0.0472)
17 tensor(0.0467)
18 tensor(0.0462)
19 tensor(0.0458)
20 tensor(0.0455)
21 tensor(0.0452)
22 tensor(0.0449)
23 tensor(0.0447)
24 tensor(0.0444)
25 tensor(0.0441)
26 tensor(0.0439)
27 tensor(0.0436)
28 tensor(0.0433)
29 tensor(0.0430)
30 tensor(0.0426)
31 tensor(0.0422)
32 tensor(0.0417)
33 tensor(0.0412)
34 tensor(0.0408)
35 tensor(0.0404)
36 tensor(0.0401)
37 tensor(0.0397)
38 tensor(0.0393)
39 tensor(0.0389)
40 tensor(0.0385)
41 tensor(0.0381)
42 tensor(0.0377)
43 tensor(0.0373)
44 tensor(0.0369)
45 tensor(0.0364)
46 tensor(0.0360)
47 tensor(0.0355)
48 tensor(0.0351)
49 tensor(0.0346)
50 tensor(0.0341)
51 tensor(0.0337)
52 tensor(0.0331)
53 tensor(0.0326)
54 tensor(0.0321)
55 tensor(0.0315)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [13]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1481.98905563]
[1409.99999118] [1523.56841469]
[1128.99997044] [1139.30523634]
[12991.99967575] [11951.74674225]
[14720.0001564] [14716.48665619]
[2938.99994564] [2740.99430943]
[6635.00014687] [7895.53642464]
[11215.99973297] [9320.13453102]
[1040.99999189] [1501.51781082]
[5040.00003624] [5130.98117065]
[9541.99988747] [8485.98801804]
[3792.99988461] [5336.20508194]
[1002.0000155] [748.071558]
[2542.00005341] [1602.12551689]
[6788.00001717] [6688.72627258]
[9386.00025749] [8231.58116722]
[1245.99996853] [1769.49110508]
[997.00000262] [732.61400366]
[708.00000262] [697.92204762]
[1435.99997544] [1414.01145315]
[2590.00006676] [2617.41862535]
[1392.99997497] [1619.18658113]
[11755.99946976] [9505.72274971]
[2007.99997377] [1349.09231567]
[2707.99998474] [2520.87729692]
[631.00000417] [700.63282824]
[652.99999881] [973.64733934]
[2492.00006247] [2656.10281944]
[594.99999416] [554.73258209]
[760.0000056] [1075.41592264]
[1179.00003028] [1054.20951796]
[1789.9999361] [172

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [14]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [847.22623825]
[628.00000334] [713.19811153]
[442.00000328] [700.39883375]
[2284.99997044] [2616.16087055]
[1040.99999189] [823.72991228]
[809.99999654] [1240.05116177]
[3049.99987316] [3649.07674026]
[720.00000596] [675.29968691]
[877.99999249] [615.63426304]
[1292.00000429] [1576.17018175]
[7056.00004578] [4816.51683807]
[4176.99999142] [5654.1961441]
[13701.00020599] [15105.37060547]
[2948.0000515] [3960.0755291]
[16732.99985504] [14517.61394501]
[1815.00000048] [2007.83846521]
[16220.00016022] [12274.92922974]
[5967.00019073] [4645.26043701]
[473.00000042] [632.33716965]
[1078.9999795] [1071.82235718]
[452.00000319] [637.39809275]
[801.99998283] [765.90419769]
[2458.99994993] [2509.01937151]
[2492.99998236] [2750.69116211]
[582.00000203] [683.5093379]
[1405.9999671] [1401.37092781]
[8715.99991035] [11690.98483276]
[7634.00004578] [6542.9805069]
[506.99999839] [634.44794321]
[665.99999094] [736.89742565]
[1815.00000048] [1484.14502954]
[707.00001383] [655.51543951]
[5

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [16]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1381.1592164302167


## Conclusão



## Playground

